In [1]:
import cv2
import face_recognition

def get_face_embeddings(image_path):
    image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(image)

    if len(face_encodings) > 0:
        return face_encodings[0]  # Return the first face encoding (assumes one face per image)
    else:
        return None

def compare_faces(known_face_encoding, unknown_face_encoding):
    # Compute the Euclidean distance between the two face embeddings
    similarity_score = face_recognition.face_distance([known_face_encoding], unknown_face_encoding)[0]

    # Normalize the similarity score to a range between 0.3 and 0.8
    # normalized_confidence = 0.3 + (similarity_score - 0) * (0.8 - 0.3) / (1 - 0)

    # return normalized_confidence
    return similarity_score

def main():
    # Load known face embedding
    known_image_path = "Known_Faces/Tolu2.jpg"
    known_face_encoding = get_face_embeddings(known_image_path)
    if known_face_encoding is None:
        print("No face detected in the known image.")
        return

    # Start webcam capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame.")
            break

        # Get face encoding of the current frame
        face_locations = face_recognition.face_locations(frame)
        if len(face_locations) > 0:
            face_encoding = face_recognition.face_encodings(frame, face_locations)[0]

            # Compare faces
            confidence = compare_faces(known_face_encoding, face_encoding)
            confidence = (1 - confidence)*100
            print(f"Confidence: {confidence:.2f}")
            
            if confidence >= 70:
              color = (0, 255, 0)
            else:
              color = (0, 0, 255)

            # Draw rectangle and confidence level on the frame
            top, right, bottom, left = face_locations[0]
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.putText(frame, f"Confidence: {confidence:.2f}", (left, bottom + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Show the frame
        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

: 

In [ ]:
import cv2
import face_recognition

# Dictionary with predefined information for known individuals
known_people = {
    "Tolu2": {
        "age": 30,
        "occupation": "Software Engineer",
        "contact": "john.doe@example.com"
    },
    "Ola": {
        "age": 25,
        "occupation": "Data Scientist",
        "contact": "jane.smith@example.com"
    }
}

def recognize_person(face_encoding, known_face_encodings):
    # Compare the face encoding with known faces
    for name, known_face_encoding in known_face_encodings.items():
        matches = face_recognition.compare_faces([known_face_encoding], face_encoding)

        if any(matches):
            return name, known_people[name]

    return None, None

def main():
    # Load known face encodings and names
    known_face_encodings = {}
    for name in known_people.keys():
        image_path = f"Known_Faces/{name.replace(' ', '_')}.jpg"  # Assuming images are stored in the 'known_faces' folder
        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings[name] = face_encoding

    # Start webcam capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame.")
            break

        # Find face locations and encodings in the frame
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            name, info = recognize_person(face_encoding, known_face_encodings)
            if name and info:
                # Calculate similarity percentage
                similarity_score = face_recognition.face_distance([known_face_encodings[name]], face_encoding)[0]
                similarity_percentage = (1 - similarity_score) * 100

                # Draw a rectangle around the face

                # Display the person's information if similarity is greater than 80%
                if similarity_percentage > 60:
                    color = (0, 255, 0)
                    # text = f"{name}\nAge: {info['age']}\nOccupation: {info['occupation']}\nContact: {info['contact']}"
                    cv2.putText(frame, name, (left, bottom + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Age: {info['age']}", (left, bottom + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Occupation: {info['occupation']}", (left, bottom + 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Contact: {info['contact']}", (left, bottom + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                else:
                    color = (0, 69, 255)
                    
                
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

                # Display the similarity percentage on the frame
                cv2.putText(frame, f"Similarity: {similarity_percentage:.2f}%", (left, bottom - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
        # Show the frame
        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
  main()

## Renaming image files in Raw Images folder

In [10]:
count = 0
raw_image_path = "Raw Images"
subs = [f for f in os.listdir(raw_image_path)]

for sub in subs:
  path = os.path.join(raw_image_path, sub)
  
  images = [f for f in os.listdir(path)]
  
  for index, image in enumerate(images, start= 1):
    new_name = f"{sub}_{index}.jpg"
    old_path = os.path.join(path, image)
    new_path = os.path.join(path, new_name)
    os.rename(old_path, new_path)
    print(f"{image} renamed to {new_name}")

photo_259@28-07-2023_15-29-00_thumb.jpg renamed to David_1.jpg
photo_260@28-07-2023_15-29-00_thumb.jpg renamed to David_2.jpg
photo_261@28-07-2023_15-29-00_thumb.jpg renamed to David_3.jpg
photo_262@28-07-2023_15-29-02_thumb.jpg renamed to David_4.jpg
photo_263@28-07-2023_15-29-02_thumb.jpg renamed to David_5.jpg
photo_264@28-07-2023_15-29-02_thumb.jpg renamed to David_6.jpg
photo_265@28-07-2023_15-29-02_thumb.jpg renamed to David_7.jpg
photo_266@28-07-2023_15-29-02_thumb.jpg renamed to David_8.jpg
photo_267@28-07-2023_15-29-02_thumb.jpg renamed to David_9.jpg
photo_268@28-07-2023_15-29-02_thumb.jpg renamed to David_10.jpg
photo_269@28-07-2023_15-29-02.jpg renamed to David_11.jpg
photo_269@28-07-2023_15-29-02_thumb.jpg renamed to David_12.jpg
photo_270@28-07-2023_15-29-02.jpg renamed to David_13.jpg
photo_270@28-07-2023_15-29-02_thumb.jpg renamed to David_14.jpg
photo_271@28-07-2023_15-29-02.jpg renamed to David_15.jpg
photo_271@28-07-2023_15-29-02_thumb.jpg renamed to David_16.jpg
pho

In [5]:
import os

count = 0

# raw_image_path = "Raw Images"

# for name in os.listdir(raw_image_path)

def rename_images_in_subfolders(root_folder):
    # Get a list of sub-folders in the root folder
    subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

    # Loop through each sub-folder
    for subfolder in subfolders:
        subfolder_path = os.path.join(root_folder, subfolder)

        # Check if the sub-folder contains image files
        image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
        if not image_files:
            continue

        # Loop through the image files and rename them with integers
        for index, image_file in enumerate(image_files, start=1):
            new_name = f"{subfolder}_{index}.jpg"  # Change the file extension if needed
            old_path = os.path.join(subfolder_path, image_file)
            new_path = os.path.join(subfolder_path, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed {image_file} to {new_name}")

# Example usage: Replace "path_to_root_folder" with the actual path to your root folder containing the sub-folders.
root_folder = "Raw Images"
rename_images_in_subfolders(root_folder)

# print(os.listdir(raw_image_path))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'David'

In [11]:
import tweepy

In [13]:
import face_recognition

# Authenticate with Twitter API
consumer_key = "Dhzcc5ewCrrg1ZYVeCUM8tSD3"
consumer_secret = "uu1ilepwkP1Wve9TlG9BzVJA3rkkaKcgTsk90AmCx8VAeUJyNF"

access_token = "1390026603225305094-HyyOPObjqOJCARhySI9oZoXOXWXZYP"
access_token_secret = "hplNyOZ2we9KrlYryxkH4fipfeRpdHO6cBOFzIyeaTVy6"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Load and encode the input face image
input_face_image = face_recognition.load_image_file("input_face.jpg")
input_face_encoding = face_recognition.face_encodings(input_face_image)[0]

# Search Twitter profiles with profile pictures
best_match_profile = None
best_similarity_score = 0.0

for tweet in tweepy.Cursor(api.search_users, q="filter:has:profile_image").items():
    profile_image_url = tweet.profile_image_url.replace("_normal", "")  # Use original resolution image
    profile_image = face_recognition.load_image_file(profile_image_url)
    profile_face_encoding = face_recognition.face_encodings(profile_image)[0]
    
    similarity_score = face_recognition.face_distance([input_face_encoding], profile_face_encoding)[0]
    
    if similarity_score > best_similarity_score:
        best_similarity_score = similarity_score
        best_match_profile = tweet

# Display the best match profile information
if best_match_profile:
    print("Best Match Profile:")
    print("Twitter Username:", best_match_profile.screen_name)
    print("Profile Image URL:", best_match_profile.profile_image_url)
    print("Similarity Score:", best_similarity_score)
else:
    print("No matching profile found.")

FileNotFoundError: [Errno 2] No such file or directory: 'input_face.jpg'